In [53]:
import torch
import torch.nn as nn
import torch.optim as optim
from PIL import Image
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.utils import save_image
import torch.nn.functional as F

In [54]:
model = models.vgg19(pretrained = True).features

In [55]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [56]:
img_size =512
l_r =0.01
epochs=200
aplha = 1
beta = 10000

In [57]:
class VGG(nn.Module):

    def __init__(self):
        super(VGG, self).__init__()
        self.chosen_features = ["0", "5", "10", "19", "28"]
        self.model = models.vgg19(pretrained=True).features[:29]
    
    def forward(self,x):
        features = []
        for layer_no,layer in enumerate(self.model):
            x=layer(x)
            if str(layer_no) in self.chosen_features:
                features.append(x)
        return features

In [58]:
def load_image(image_name):
    image = Image.open(image_name)
    image = loader(image).unsqueeze(0)
    return image.to(device)

In [59]:
loader = transforms.Compose(
    [
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor(),
        #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

In [60]:
original_img= load_image('conta.jpg')
style_img = load_image('style.jpg')

In [61]:
generated = original_img.clone().requires_grad_(True)
model = VGG().to(device).eval()

In [62]:
def gram_matrix(tensor):
    _, c, h, w = tensor.size()
    features = tensor.view(c, h * w)
    G = torch.mm(features, features.t())
    return G

In [63]:
def get_style_loss(target, style):
  G = gram_matrix(target)
  S = gram_matrix(style)
  style_loss = F.mse_loss(G, S)
  return style_loss

In [64]:
def get_content_loss(target, content):
  return F.mse_loss(target, content)

In [65]:
optimizer = optim.Adam([generated],lr = l_r)
for i in range(epochs):
    generated_features = model(generated)
    original_img_features = model(original_img)
    style_features = model(style_img)
    
    style_loss=0
    gen_loss=0

    for gen,con,sty in zip(generated_features, original_img_features, style_features):

        content_loss = get_content_loss(gen,con)
        style_loss = get_style_loss(gen,sty)

    total_loss = aplha* content_loss + beta * style_loss
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()

    if epochs % 200 == 0:
        print(total_loss)
        save_image(generated, "generated.png")


tensor(1.6092e+11, device='cuda:0', grad_fn=<AddBackward0>)
tensor(8.8068e+10, device='cuda:0', grad_fn=<AddBackward0>)
tensor(6.0138e+10, device='cuda:0', grad_fn=<AddBackward0>)
tensor(5.3900e+10, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.7561e+10, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.6278e+10, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.8856e+10, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.6459e+10, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.2397e+10, device='cuda:0', grad_fn=<AddBackward0>)
tensor(9.8174e+09, device='cuda:0', grad_fn=<AddBackward0>)
tensor(9.2499e+09, device='cuda:0', grad_fn=<AddBackward0>)
tensor(7.2355e+09, device='cuda:0', grad_fn=<AddBackward0>)
tensor(6.4589e+09, device='cuda:0', grad_fn=<AddBackward0>)
tensor(5.9592e+09, device='cuda:0', grad_fn=<AddBackward0>)
tensor(4.9903e+09, device='cuda:0', grad_fn=<AddBackward0>)
tensor(4.7401e+09, device='cuda:0', grad_fn=<AddBackward0>)
tensor(4.1553e+09, device='cuda:0', grad